In [1]:
import io
import re
import numpy as np
import pandas as pd
import numpy.linalg as lin
import warnings 
from gensim.models import Word2Vec
import pickle
from sklearn.cluster import KMeans
warnings.filterwarnings('ignore')

In [2]:
alphabets=['ऄ','अ','आ','इ','ई','उ','ऊ','ऋ','ऌ','ऍ','ऎ','ए','ऐ','ऑ','ऒ','ओ','औ','क','ख','ग','घ','ङ','च','छ','ज','झ','ञ','ट',
'ठ','ड','ढ','ण','त','थ','द','ध','न','ऩ','प','फ','ब','भ','म','य','र','ऱ','ल','ळ','ऴ','व','श','ष','स','ह']

matra=['ऀ','ँ','ं','ः','ऺ','ऻ','़','ा','ि','ी','ु','ू','ृ','ॄ','ॅ','ॆ','े','ै','ॉ','ॊ','ो','ौ','्','ॎ','ॏ','ॕ','ॖ','ॗ']

In [3]:
stopwords=["अंदर","अत","अदि","अप","अपना","अपनि","अपनी","अपने","अभि","अभी","आदि","आप","इंहिं","इंहें","इंहों","इतयादि","इत्यादि","इन","इनका","इन्हीं","इन्हें","इन्हों","इस","इसका","इसकि","इसकी","इसके","इसमें","इसि","इसी","इसे","उंहिं","उंहें","उंहों","उन","उनका","उनकि","उनकी","उनके","उनको","उन्हीं","उन्हें","उन्हों","उस","उसके","उसि","उसी","उसे","एक","एवं","एस","एसे","ऐसे","ओर","और","कइ","कई","कर","करता","करते","करना","करने","करें","कहते","कहा","का","काफि","काफ़ी","कि","किंहें","किंहों","कितना","किन्हें","किन्हों","किया","किर","किस","किसि","किसी","किसे","की","कुछ","कुल","के","को","कोइ","कोई","कोन","कोनसा","कौन","कौनसा","गया","घर","जब","जहाँ","जहां","जा","जिंहें","जिंहों","जितना","जिधर","जिन","जिन्हें","जिन्हों","जिस","जिसे","जीधर","जेसा","जेसे","जैसा","जैसे","जो","तक","तब","तरह","तिंहें","तिंहों","तिन","तिन्हें","तिन्हों","तिस","तिसे","तो","था","थि","थी","थे","दबारा","दवारा","दिया","दुसरा","दुसरे","दूसरे","दो","द्वारा","न","नहिं","नहीं","ना","निचे","निहायत","नीचे","ने","पर","पहले","पुरा","पूरा","पे","फिर","बनि","बनी","बहि","बही","बहुत","बाद","बाला","बिलकुल","भि","भितर","भी","भीतर","मगर","मानो","मे","में","यदि","यह","यहाँ","यहां","यहि","यही","या","यिह","ये","रखें","रवासा","रहा","रहे","ऱ्वासा","लिए","लिये","लेकिन","व","वगेरह","वरग","वर्ग","वह","वहाँ","वहां","वहिं","वहीं","वाले","वुह","वे","वग़ैरह","संग","सकता","सकते","सबसे","सभि","सभी","साथ","साबुत","साभ","सारा","से","सो","हि","ही","हुअ","हुआ","हुइ","हुई","हुए","हे","हें","है","हैं","हो","होता","होति","होती","होते","होना","होने"]
delimiters=["।","?","!"]

In [4]:
cbow=Word2Vec.load('cbow/hi-d100-m2-cbow.model')

In [5]:
file=open('sentence.pkl','rb')
verse_embeddings_sentence=pickle.load(file)
file.close()
file=open('max.pkl','rb')
verse_embeddings_max=pickle.load(file)
file.close()
file=open('mean.pkl','rb')
verse_embeddings_mean=pickle.load(file)
file.close()

In [6]:
file=open('whole.pkl','rb')
verse_embeddings_whole=pickle.load(file)
file.close()

In [7]:
def remove_special_characters(text):
    string=''
    for i in text:
        if i in alphabets+matra+delimiters:
            string=string+i
        else:
            string=string+" "
    return " ".join(string.split())
def sent_tokenize(text):
    text=remove_special_characters(text)
    regexPattern = '|'.join(map(re.escape, delimiters))
    sentences=re.split(regexPattern,text)
    return [sent.strip() for sent in sentences if len(sent.strip())>0]

In [8]:
def word_embedding(word):
    try:
        return cbow.wv[word]
    except:
        return np.array([0.0]*100)

In [9]:
def cosine_similarity(a,b):
    try:
        return a.dot(b)/(lin.norm(a)*lin.norm(b))
    except:
        return -1.0

In [10]:
def ed(a,b):
    return lin.norm(a-b)

In [11]:
def sentence_embedding(sentence):
    sentence=remove_special_characters(sentence)
    words=sentence.split()
    words=[word  for word in words if word not in stopwords]
    if len(words)>0:
        sentence_embedding=[word_embedding(word) for word in words]
        return np.array(list(map(lambda x: sum(x)/len(x), zip(*sentence_embedding))))
    return np.array([0.0]*100)

In [12]:
data=pd.read_csv('../hindi.csv')

In [13]:
query='भौतिक जगत में माया का आधिपत्य होता है और श्रीकृष्ण ने इसे आठवें अध्याय के 15वें श्लोक में अस्थायी और दुखों से भरा हुआ कह कर परिभाषित किया है। इस प्रकार भौतिक शक्ति माया की तुलना अंधकार से की जाती है।'
query_embedding=sentence_embedding(query)
query_embedding_list=query_embedding.reshape(1,-1).astype(np.float64)

# Max Strategy

## Cosine Similarity

In [14]:
def max_cosine(query_embedding,counter=10):
    scoring=[cosine_similarity(query_embedding,i) for i in verse_embeddings_max]
    verse_index=np.argsort(scoring)[-1::-1][:counter]
    return verse_index

In [15]:
verse_index=max_cosine(query_embedding)

In [16]:
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
333,9,1,श्री भगवानुवाच\n\nइदं तु ते गुह्यतमं प्रवक्ष्य...,इस विषय पर प्रारम्भ में ही श्रीकृष्ण उनके इन उ...
183,4,27,सर्वाणीन्द्रियकर्माणि प्राणकर्माणि चापरे।\n\nआ...,कुछ योगी जो विवेकबुद्धि या ज्ञानयोग के मार्ग क...
216,5,18,विद्याविनयसंपन्ने ब्राह्मणे गवि हस्तिनि।\n\nशु...,जब हम ज्ञान के परिप्रेक्ष्य में चीजों को देखते...
93,2,52,यदा ते मोहकलिलं बुद्धिर्व्यतितरिष्यति।\n\nतदा ...,श्रीकृष्ण ने पहले के श्लोकों मे स्पष्ट किया है...
215,5,17,तद्बुद्धयस्तदात्मानस्तन्निष्ठास्तत्परायणाः।\n\...,जिस प्रकार अज्ञानता के कारण कोई 'संसार' या 'ज...
561,15,17,उत्तमः पुरुषस्त्वन्यः परमात्मेत्युदाहृतः।यो लो...,संसार और आत्मा का निरूपण करने के पश्चात अब श्र...
273,6,46,तपस्विभ्योऽधिको योगी ज्ञानिभ्योऽपि मतोऽधिकः।\n...,तपस्वी वह है जो स्वेच्छा से वैराग्य स्वीकार कर...
336,9,4,मया ततमिदं सर्वं जगदव्यक्तमूर्तिना।\n\nमत्स्था...,वैदिक दर्शन इस अवधारणा को स्वीकार नहीं करता क...
419,11,11,दिव्यमाल्याम्बरधरं दिव्यगन्धानुलेपनम्।\n\nसर्व...,संजय अनेक और अनंत शब्दों का प्रयोग कर श्रीकृष...
418,11,10,अनेकवक्त्रनयनमनेकाद्भुतदर्शनम्।\n\nअनेकदिव्याभ...,संजय अनेक और अनंत शब्दों का प्रयोग कर श्रीकृष...


## Euclidean Distance

In [17]:
def max_ed(query_embedding,counter=10):
    scoring=[ed(query_embedding,i) for i in verse_embeddings_max]
    verse_index=np.argsort(scoring)[:counter]
    return verse_index

In [18]:
verse_index=max_ed(query_embedding)

In [19]:
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
654,18,38,विषयेन्द्रियसंयोगाद्यत्तदग्रेऽमृतोपमम्।परिणामे...,राजसिक सुख एक रोमांच की भांति है जो इन्द्रियों...
93,2,52,यदा ते मोहकलिलं बुद्धिर्व्यतितरिष्यति।\n\nतदा ...,श्रीकृष्ण ने पहले के श्लोकों मे स्पष्ट किया है...
459,11,51,अर्जुन उवाच\n\nदृष्ट्वेदं मानुषं रूपं तवसौम्यं...,श्रीकृष्ण के मनमोहक सुन्दर दो भुजाओं वाले रूप ...
409,11,1,अर्जुन उवाच\n\nमदनुग्रहाय परमं गुह्यमध्यात्मसं...,अर्जुन भगवान श्रीकृष्ण की विभूतियों को सुनकर औ...
413,11,5,श्री भगवानुवाच\n\nपश्य मे पार्थ रूपाणि शतशोऽथ ...,अर्जुन की प्रार्थना सुनकर श्रीकृष्ण अब उससे कह...
430,11,22,रुद्रादित्या वसवो ये च साध्या\n\nविश्वेऽश्िवनौ...,ये सभी महान विभूतियाँ भगवान की शक्ति से ही अप...
309,8,5,अन्तकाले च मामेव स्मरन्मुक्त्वा कलेवरम्।\n\nयः...,अगले श्लोक में श्रीकृष्ण इस सिद्धान्त की व्या...
50,2,9,सञ्जय उवाच\n\nएवमुक्त्वा हृषीकेशं गुडाकेशः परन...,दूरदर्शी संजय ने धृतराष्ट्र को सुनाए जा रहे वृ...
607,17,19,मूढग्राहेणात्मनो यत्पीडया क्रियते तपः।परस्योत्...,मूढग्राहेणात्मनो शब्द ऐसे व्यक्तियों के लिए प्...
272,6,45,प्रयत्नाद्यतमानस्तु योगी संशुद्धकिल्बिषः।\n\nअ...,पिछले जन्म के संचित अभ्यास आध्यात्मिक उन्नति क...


# Mean Pooling

## Cosine Similarity

In [20]:
def mean_cosine(query_embedding,counter=10):
    scoring=[cosine_similarity(query_embedding,i) for i in verse_embeddings_mean]
    verse_index=np.argsort(scoring)[-1::-1][:counter]
    return verse_index

In [21]:
verse_index=mean_cosine(query_embedding)

In [22]:
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
250,6,23,तं विद्याद् दुःखसंयोगवियोगं योगसंज्ञितम्।\n\nस...,भौतिक जगत में माया का आधिपत्य होता है और श्रीक...
113,2,72,एषा ब्राह्मी स्थितिः पार्थ नैनां प्राप्य विमुह...,ब्रह्म का अर्थ भगवान तथा स्थिति का अर्थ भगवद्...
322,8,18,अव्यक्ताद्व्यक्तयः सर्वाः प्रभवन्त्यहरागमे।\n\...,ब्रह्माण्ड के अद्भुत ब्रह्माण्डीय खेल में विभि...
551,15,7,ममैवांशो जीवलोके जीवभूतः सनातनः।मनःषष्ठानीन्द्...,श्रीकृष्ण ने पहले यह समझाया था कि जो उनके लोक ...
340,9,8,प्रकृतिं स्वामवष्टभ्य विसृजामि पुनः पुनः।\n\nभ...,पिछले दो श्लोकों में श्रीकृष्ण ने यह व्याख्या ...
339,9,7,सर्वभूतानि कौन्तेय प्रकृतिं यान्ति मामिकाम्।\n...,पिछले दो श्लोकों में श्रीकृष्ण ने यह व्याख्या ...
336,9,4,मया ततमिदं सर्वं जगदव्यक्तमूर्तिना।\n\nमत्स्था...,वैदिक दर्शन इस अवधारणा को स्वीकार नहीं करता क...
324,8,20,परस्तस्मात्तु भावोऽन्योऽव्यक्तोऽव्यक्तात्सनातन...,लौकिक संसारों और उनके अस्थायित्व पर अपना प्रकट...
279,7,5,अपरेयमितस्त्वन्यां प्रकृतिं विद्धि मे पराम्।\n...,श्रीकृष्ण अब पूर्णतया भौतिक ज्ञान के क्षेत्र स...
506,13,24,य एवं वेत्ति पुरुषं प्रकृतिं च गुणैःसह।सर्वथा ...,अज्ञानता आत्मा को दुर्दशा की ओर ले जाती है। भग...


## Euclidean Distance

In [23]:
def mean_ed(query_embedding,counter=10):
    scoring=[ed(query_embedding,i) for i in verse_embeddings_mean]
    verse_index=np.argsort(scoring)[:counter]
    return verse_index

In [24]:
verse_index=mean_ed(query_embedding)

In [25]:
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
250,6,23,तं विद्याद् दुःखसंयोगवियोगं योगसंज्ञितम्।\n\nस...,भौतिक जगत में माया का आधिपत्य होता है और श्रीक...
551,15,7,ममैवांशो जीवलोके जीवभूतः सनातनः।मनःषष्ठानीन्द्...,श्रीकृष्ण ने पहले यह समझाया था कि जो उनके लोक ...
113,2,72,एषा ब्राह्मी स्थितिः पार्थ नैनां प्राप्य विमुह...,ब्रह्म का अर्थ भगवान तथा स्थिति का अर्थ भगवद्...
506,13,24,य एवं वेत्ति पुरुषं प्रकृतिं च गुणैःसह।सर्वथा ...,अज्ञानता आत्मा को दुर्दशा की ओर ले जाती है। भग...
278,7,4,भूमिरापोऽनलो वायुः खं मनो बुद्धिरेव च।\n\nअहङ्...,"प्राकृत शक्ति से रचित संसार विचित्र, जटिल और अ..."
320,8,16,आब्रह्मभुवनाल्लोकाः पुनरावर्तिनोऽर्जुन।\n\nमाम...,वैदिक ग्रंथों में पृथ्वी लोक से नीचे सात लोकों...
279,7,5,अपरेयमितस्त्वन्यां प्रकृतिं विद्धि मे पराम्।\n...,श्रीकृष्ण अब पूर्णतया भौतिक ज्ञान के क्षेत्र स...
336,9,4,मया ततमिदं सर्वं जगदव्यक्तमूर्तिना।\n\nमत्स्था...,वैदिक दर्शन इस अवधारणा को स्वीकार नहीं करता क...
281,7,7,मत्तः परतरं नान्यत्किञ्चिदस्ति धनञ्जय।\n\nमयि ...,यह कहने के पश्चात कि वे सबके मूल और सभी अस्तित...
385,10,19,श्री भगवानुवाच\n\nहन्त ते कथयिष्यामि दिव्या ह्...,अमरकोष (अति प्रतिष्ठित प्राचीन संस्कृत शब्द को...


# Line Matching

## Cosine Similarity

In [26]:
def line_cosine(query_embeddings,counter=10):
    scoring=[]
    for embeddings in verse_embeddings_sentence:
        score=[cosine_similarity(query_embedding,embed) for embed in embeddings]
        scoring.append(max(score))
    verse_index=np.argsort(scoring)[-1::-1][:counter]
    return verse_index

In [27]:
verse_index=line_cosine(query_embedding)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
250,6,23,तं विद्याद् दुःखसंयोगवियोगं योगसंज्ञितम्।\n\nस...,भौतिक जगत में माया का आधिपत्य होता है और श्रीक...
408,10,42,अथवा बहुनैतेन किं ज्ञातेन तवार्जुन।\n\nविष्टभ्...,श्रीकृष्ण का उपर्युक्त कथन यह इंगित करता है कि...
561,15,17,उत्तमः पुरुषस्त्वन्यः परमात्मेत्युदाहृतः।यो लो...,संसार और आत्मा का निरूपण करने के पश्चात अब श्र...
549,15,5,निर्मानमोहा जितसङ्गदोषा अध्यात्मनित्या विनि...,"श्रीकृष्ण अब बताते हैं कि इस वृक्ष के आधार, जो..."
307,8,3,श्री भगवानुवाच\n\nअक्षरं ब्रह्म परमं स्वभावोऽध...,श्रीकृष्ण कहते हैं कि परम सत्ता को ब्रह्म कहा...
374,10,8,अहं सर्वस्य प्रभवो मत्तः सर्वं प्रवर्तते।\n\nइ...,श्रीकृष्ण इस श्लोक का आरम्भ अहं सर्वस्य प्रभवो...
465,12,2,श्री भगवानुवाचमय्यावेश्य मनो ये मां नित्ययुक्त...,भगवान की निकटता विभिन्न कोणों से अनुभव की जा स...
57,2,16,नासतो विद्यते भावो नाभावो विद्यते सतः।\n\nउभयो...,श्वेताश्वतरोपनिषद् के अनुसार संसार में तीन तत्...
524,14,7,रजो रागात्मकं विद्धि तृष्णासङ्गसमुद्भवम्।तन्नि...,श्रीकृष्ण अब रजोगुण की कार्यविधि को समझाते हुए...
356,9,24,अहं हि सर्वयज्ञानां भोक्ता च प्रभुरेव च।\n\nन ...,श्रीकृष्ण अब देवताओं की पूजा करने के दोषों को...


## Euclidean Distance

In [28]:
def line_ed(query_embeddings,counter=10):
    scoring=[]
    for embeddings in verse_embeddings_sentence:
        score=[ed(query_embedding,embed) for embed in embeddings]
        scoring.append(max(score))
    verse_index=np.argsort(scoring)[:counter]
    return verse_index

In [29]:
verse_index=line_ed(query_embedding)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
93,2,52,यदा ते मोहकलिलं बुद्धिर्व्यतितरिष्यति।\n\nतदा ...,श्रीकृष्ण ने पहले के श्लोकों मे स्पष्ट किया है...
430,11,22,रुद्रादित्या वसवो ये च साध्या\n\nविश्वेऽश्िवनौ...,ये सभी महान विभूतियाँ भगवान की शक्ति से ही अप...
309,8,5,अन्तकाले च मामेव स्मरन्मुक्त्वा कलेवरम्।\n\nयः...,अगले श्लोक में श्रीकृष्ण इस सिद्धान्त की व्या...
654,18,38,विषयेन्द्रियसंयोगाद्यत्तदग्रेऽमृतोपमम्।परिणामे...,राजसिक सुख एक रोमांच की भांति है जो इन्द्रियों...
457,11,49,मा ते व्यथा मा च विमूढभावो\n\nदृष्ट्वा रूपं घो...,श्रीकृष्ण निरन्तर अर्जुन को धीरज बंधाते हुए कह...
459,11,51,अर्जुन उवाच\n\nदृष्ट्वेदं मानुषं रूपं तवसौम्यं...,श्रीकृष्ण के मनमोहक सुन्दर दो भुजाओं वाले रूप ...
607,17,19,मूढग्राहेणात्मनो यत्पीडया क्रियते तपः।परस्योत्...,मूढग्राहेणात्मनो शब्द ऐसे व्यक्तियों के लिए प्...
272,6,45,प्रयत्नाद्यतमानस्तु योगी संशुद्धकिल्बिषः।\n\nअ...,पिछले जन्म के संचित अभ्यास आध्यात्मिक उन्नति क...
160,4,4,अर्जुन उवाच\n\nअपरं भवतो जन्म परं जन्म विवस्वत...,अर्जुन भगवान श्रीकृष्ण के कथनों में घटनाओं की ...
37,1,41,अधर्माभिभवात्कृष्ण प्रदुष्यन्ति कुलस्त्रियः।\n...,वैदिक सभ्यता में समाज में स्त्रियों को उच्च स...


# Whole Part

## Cosine Similarity

In [30]:
def whole_cosine(query_embedding,counter=10):
    scoring=[cosine_similarity(query_embedding,i) for i in verse_embeddings_whole]
    verse_index=np.argsort(scoring)[-1::-1][:counter]
    return verse_index

In [31]:
verse_index=whole_cosine(query_embedding)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
250,6,23,तं विद्याद् दुःखसंयोगवियोगं योगसंज्ञितम्।\n\nस...,भौतिक जगत में माया का आधिपत्य होता है और श्रीक...
322,8,18,अव्यक्ताद्व्यक्तयः सर्वाः प्रभवन्त्यहरागमे।\n\...,ब्रह्माण्ड के अद्भुत ब्रह्माण्डीय खेल में विभि...
339,9,7,सर्वभूतानि कौन्तेय प्रकृतिं यान्ति मामिकाम्।\n...,पिछले दो श्लोकों में श्रीकृष्ण ने यह व्याख्या ...
340,9,8,प्रकृतिं स्वामवष्टभ्य विसृजामि पुनः पुनः।\n\nभ...,पिछले दो श्लोकों में श्रीकृष्ण ने यह व्याख्या ...
324,8,20,परस्तस्मात्तु भावोऽन्योऽव्यक्तोऽव्यक्तात्सनातन...,लौकिक संसारों और उनके अस्थायित्व पर अपना प्रकट...
336,9,4,मया ततमिदं सर्वं जगदव्यक्तमूर्तिना।\n\nमत्स्था...,वैदिक दर्शन इस अवधारणा को स्वीकार नहीं करता क...
113,2,72,एषा ब्राह्मी स्थितिः पार्थ नैनां प्राप्य विमुह...,ब्रह्म का अर्थ भगवान तथा स्थिति का अर्थ भगवद्...
307,8,3,श्री भगवानुवाच\n\nअक्षरं ब्रह्म परमं स्वभावोऽध...,श्रीकृष्ण कहते हैं कि परम सत्ता को ब्रह्म कहा...
279,7,5,अपरेयमितस्त्वन्यां प्रकृतिं विद्धि मे पराम्।\n...,श्रीकृष्ण अब पूर्णतया भौतिक ज्ञान के क्षेत्र स...
551,15,7,ममैवांशो जीवलोके जीवभूतः सनातनः।मनःषष्ठानीन्द्...,श्रीकृष्ण ने पहले यह समझाया था कि जो उनके लोक ...


## Euclidean Distance

In [32]:
def whole_ed(query_embedding,counter=10):
    scoring=[cosine_similarity(query_embedding,i) for i in verse_embeddings_whole]
    verse_index=np.argsort(scoring)[:counter]
    return verse_index

In [33]:
verse_index=whole_ed(query_embedding)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
486,13,4,तत्क्षेत्रं यच्च यादृक् च यद्विकारि यतश्च यत्।...,श्रीकृष्ण अब स्वयं कई प्रश्न खड़े करते हैं और ...
22,1,26,तत्रापश्यत्स्थितान्पार्थः पितृ़नथ पितामहान्।\n...,अर्जुन युद्धभूमि में अपने सभी सम्बंधियों को दे...
8,1,11,अयनेषु च सर्वेषु यथाभागमवस्थिताः।\n\nभीष्ममेवा...,दुर्योधन यह समझ गया था कि भीष्म पितामह मानसिक ...
443,11,35,सञ्जय उवाच\n\nएतच्छ्रुत्वा वचनं केशवस्य\n\nकृत...,यहाँ अर्जुन को 'किरीटी' कह कर संबोधित किया गया...
597,17,9,कट्वम्ललवणात्युष्णतीक्ष्णरूक्षविदाहिनः।आहारा र...,"जब शाकाहारी भोजनों को अत्यधिक मीर्च, शर्करा, न..."
660,18,44,कृषिगौरक्ष्यवाणिज्यं वैश्यकर्म स्वभावजम्।परिचर...,वैश्यों में रजोगुण की प्रधानता के साथ तमोगुण म...
659,18,43,शौर्यं तेजो धृतिर्दाक्ष्यं युद्धे चाप्यपलायनम्...,क्षत्रिय उन्हें कहते थे जिनमें सत्वगुण सहित रज...
17,1,20,अथ व्यवस्थितान् दृष्ट्वा धार्तराष्ट्रान्कपिध्व...,इस श्लोक में अर्जुन को कपि-ध्वज कह कर संबोधित ...
10,1,13,ततः शङ्खाश्च भेर्यश्च पणवानकगोमुखाः।\n\nसहसैवा...,युद्ध के लिए भीष्म पितामह के तीव्र उत्साह को द...
3,1,4,अत्र शूरा महेष्वासा भीमार्जुनसमा युधि।\n\nयुयु...,अपने सम्मख संकट को मंडराते देखकर दुर्योधन को प...


# Clustering

## Max Pooling

In [34]:
kmeans_max = KMeans(n_clusters=60, random_state=0).fit(verse_embeddings_max)

In [35]:
def kmeans_max_cluster(query_embedding_list):
    label=kmeans_max.predict(query_embedding_list)[0]
    verse_index=[i for i in range(len(data)) if kmeans_max.labels_[i]==label]
    return verse_index

In [36]:
verse_index=kmeans_max_cluster(query_embedding_list)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
23,1,27,श्वशुरान्सुहृदश्चैव सेनयोरुभयोरपि।\n\nतान्समीक...,अपने बंधु बान्धवों को युद्ध स्थल पर एकत्रित दे...
50,2,9,सञ्जय उवाच\n\nएवमुक्त्वा हृषीकेशं गुडाकेशः परन...,दूरदर्शी संजय ने धृतराष्ट्र को सुनाए जा रहे वृ...
52,2,11,श्री भगवानुवाच\n\nअशोच्यानन्वशोचस्त्वं प्रज्ञा...,इस श्लोक के साथ श्रीकृष्ण अपने दिव्य उपदेश का ...
132,3,19,तस्मादसक्तः सततं कार्यं कर्म समाचर।\n\nअसक्तो ...,श्लोक 3.8-3.16 में श्रीकृष्ण पुरजोर आह्वान करत...
208,5,10,ब्रह्मण्याधाय कर्माणि सङ्गं त्यक्त्वा करोति यः...,हिन्दू और बौद्ध दोनों धर्मों के धार्मिक ग्रंथ ...
362,9,30,अपि चेत्सुदुराचारो भजते मामनन्यभाक्।\n\nसाधुरे...,परम प्रभु के प्रति की जाने वाली भक्ति में इतन...
366,9,34,मन्मना भव मद्भक्तो मद्याजी मां नमस्कुरु।\n\nमा...,इस पूरे अध्याय में भक्ति मार्ग का अनुसरण करने...
367,10,1,श्री भगवानुवाच\n\nभूय एव महाबाहो श्रृणु मे परम...,श्रीकृष्ण उनकी महिमा सुनने के लिए अर्जुन द्वा...
409,11,1,अर्जुन उवाच\n\nमदनुग्रहाय परमं गुह्यमध्यात्मसं...,अर्जुन भगवान श्रीकृष्ण की विभूतियों को सुनकर औ...
410,11,2,भवाप्ययौ हि भूतानां श्रुतौ विस्तरशो मया।\n\nत्...,अर्जुन समस्त भौतिक अभिव्यक्तियों के उत्पन्न औ...


## Mean Pooling

In [37]:
kmeans_mean = KMeans(n_clusters=60, random_state=0).fit(verse_embeddings_mean)

In [38]:
def kmeans_mean_cluster(query_embedding_list):
    label=kmeans_mean.predict(query_embedding_list)[0]
    verse_index=[i for i in range(len(data)) if kmeans_mean.labels_[i]==label]
    return verse_index

In [39]:
verse_index=kmeans_mean_cluster(query_embedding_list)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
54,2,13,देहिनोऽस्मिन्यथा देहे कौमारं यौवनं जरा।\n\nतथा...,श्रीकृष्ण सटीक तर्क के साथ आत्मा के एक जन्म से...
256,6,29,सर्वभूतस्थमात्मानं सर्वभूतानि चात्मनि।\n\nईक्ष...,भारत में दीपावली के उत्सव में दुकानों में कई प...
278,7,4,भूमिरापोऽनलो वायुः खं मनो बुद्धिरेव च।\n\nअहङ्...,"प्राकृत शक्ति से रचित संसार विचित्र, जटिल और अ..."
389,10,23,रुद्राणां शङ्करश्चास्मि वित्तेशो यक्षरक्षसाम्।...,"रुद्र, भगवान शिव के ग्यारह स्वरूप हैं-हर, बहुर..."
399,10,33,अक्षराणामकारोऽस्मि द्वन्द्वः सामासिकस्य च।\n\n...,संस्कृत वर्णमाला के सभी वर्ण आधे 'अ' अक्षर के ...
407,10,41,यद्यद्विभूतिमत्सत्त्वं श्रीमदूर्जितमेव वा।\n\n...,स्पीकर द्वारा प्रवाहित हो रही बिजली ध्वनि उत्प...
506,13,24,य एवं वेत्ति पुरुषं प्रकृतिं च गुणैःसह।सर्वथा ...,अज्ञानता आत्मा को दुर्दशा की ओर ले जाती है। भग...
516,13,34,यथा प्रकाशयत्येकः कृत्स्नं लोकमिमं रविः।क्षेत्...,यद्यपि आत्मा चेतना के साथ जिस शरीर में रहती है...
522,14,5,सत्त्वं रजस्तम इति गुणाः प्रकृतिसंभवाः।निबध्नन...,यह समझाने के पश्चात कि सभी जीवन रूप पुरुष और प...
612,17,24,तस्मादोमित्युदाहृत्य यज्ञदानतपःक्रियाः।प्रवर्त...,ओम् शब्दांश भगवान के निरोकार रूप की एक प्रतीका...


## Whole

In [40]:
kmeans_whole = KMeans(n_clusters=60, random_state=0).fit(verse_embeddings_whole)

In [41]:
def kmeans_whole_cluster(query_embedding_list):
    label=kmeans_whole.predict(query_embedding_list)[0]
    verse_index=[i for i in range(len(data)) if kmeans_whole.labels_[i]==label]
    return verse_index

In [42]:
verse_index=kmeans_whole_cluster(query_embedding_list)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
61,2,20,न जायते म्रियते वा कदाचि\n\nन्नायं भूत्वा भवित...,इस श्लोक में आत्मा की शाश्वतता को सिद्ध किया ग...
113,2,72,एषा ब्राह्मी स्थितिः पार्थ नैनां प्राप्य विमुह...,ब्रह्म का अर्थ भगवान तथा स्थिति का अर्थ भगवद्...
211,5,13,सर्वकर्माणि मनसा संन्यस्यास्ते सुखं वशी।\n\nनव...,श्रीकृष्ण अब शरीर की तुलना नौ द्वारों को खोलने...
258,6,31,सर्वभूतस्थितं यो मां भजत्येकत्वमास्थितः।\n\nसर...,भगवान संसार में सर्वत्र सर्वव्यापक है। वे परमा...
336,9,4,मया ततमिदं सर्वं जगदव्यक्तमूर्तिना।\n\nमत्स्था...,वैदिक दर्शन इस अवधारणा को स्वीकार नहीं करता क...
338,9,6,यथाऽऽकाशस्थितो नित्यं वायुः सर्वत्रगो महान्।\n...,श्रीकृष्ण ने चौथे श्लोक से छठे श्लोक तक 'मत्स्...
386,10,20,अहमात्मा गुडाकेश सर्वभूताशयस्थितः।\n\nअहमादिश्...,श्रीकृष्ण स्पष्ट कहते हैं कि वे आत्मा से दूर न...
388,10,22,वेदानां सामवेदोऽस्मि देवानामस्मि वासवः।\n\nइन्...,"चार प्रकार के वेदों के नाम ऋग्वेद, यजुर्वेद, ..."
465,12,2,श्री भगवानुवाचमय्यावेश्य मनो ये मां नित्ययुक्त...,भगवान की निकटता विभिन्न कोणों से अनुभव की जा स...
499,13,17,अविभक्तं च भूतेषु विभक्तमिव च स्थितम्।भूतभर्तृ...,भगवान के स्वरूप में उनकी विभिन्न शक्तियाँ सम्म...
